# Run Dense reconstruction

This notebook will guide you for running a dense reconstruction with LOFTR or RoMa starting from an existing reconstruction (e.g., carried out with SuperPoint+LightGlue that are more robust for image orientation).

This notebook will perform the dense matching with RoMa and it uses pycolmap to triangulate the points from the existing camera poses using pycolmap. To build a sparse reconstruction with SuperPoint+LightGlue, you can use the notebook `sfm_pipeline.ipynb`.


In [1]:
import pycolmap

import deep_image_matching as dim
from deep_image_matching.triangulation import db_from_existing_poses
from deep_image_matching.utils import OutputCapture

logger = dim.setup_logger("dim")

# Define the paraemters for the dense matching
params = {
    "dir": "../assets/example_cyprus",
    "pipeline": "roma",
    "config_file": "../assets/example_cyprus/config_roma.yaml",
    "strategy": "matching_lowres",
    "quality": "medium",
    "tiling": "none",
    "skip_reconstruction": True,
    "force": True,
    "camera_options": "../assets/example_cyprus/cameras.yaml",
    "openmvg": None,
}

# Build the configuration object
config = dim.Config(params)

Deep Image Matching loaded in 3.127 seconds.
Using a custom configuration file: /home/francesco/phd/deep-image-matching/assets/example_cyprus/config_roma.yaml
Config general:
{'camera_options': '../assets/example_cyprus/cameras.yaml',
 'db_path': None,
 'geom_verification': <GeometricVerification.NONE: 0>,
 'graph': True,
 'gv_confidence': 0.9999,
 'gv_threshold': 3,
 'image_dir': PosixPath('../assets/example_cyprus/images'),
 'matching_strategy': 'matching_lowres',
 'min_inlier_ratio_per_pair': 0.2,
 'min_inliers_per_pair': 5,
 'min_matches_per_tile': 3,
 'openmvg_conf': None,
 'output_dir': PosixPath('../assets/example_cyprus/results_roma_matching_lowres_quality_medium'),
 'overlap': None,
 'pair_file': PosixPath('../assets/example_cyprus/results_roma_matching_lowres_quality_medium/pairs.txt'),
 'preselection_pipeline': 'roma',
 'quality': <Quality.MEDIUM: 2>,
 'refine_intrinsics': False,
 'retrieval': None,
 'skip_reconstruction': True,
 'tile_overlap': 50,
 'tile_preselection_size'

Define the parameters for building the dense reconstruction starting from the known camera poses computed previously.


In [2]:
# Define project directory
root_path = config.general["output_dir"].parent
image_dir = config.general["image_dir"]

# Path to the pre-computed COLMAP reconstuction with the knwon poses (set your own path)
sfm_path = root_path / "results_superpoint+lightglue_matching_lowres_quality_medium"
sfm_db_path = sfm_path / "database.db"
sfm_rec_path = sfm_path / "reconstruction"

# Path to the dense matching results to be triangulated
dense_path = config.general["output_dir"]
features_h5 = dense_path / "features.h5"
matches_h5 = dense_path / "matches.h5"
pair_file = dense_path / "pairs.txt"

# Path to the output for the dense matching
dense_db_path = dense_path / "database_dense.db"
model_path = dense_path / "dense_model"
model_path.mkdir(exist_ok=True, parents=True)

# Do geometric verification of the dense features (computing the epipolar error)
do_geometric_verification = True
max_error = 6

# Define trinagulation parameters
triang_min_angle = 0.1

In [3]:
# Initialize ImageMatcher class
matcher = dim.ImageMatcher(config)

# Run image matching
feature_path, match_path = matcher.run()

# Export in colmap format
database_path = config.general["output_dir"] / "database.db"
dim.io.export_to_colmap(
    img_dir=config.general["image_dir"],
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_config_path=config.general["camera_options"],
)

2024-04-13 13:38:19 | [INFO    ] RoMa always use a coarse resolution of 860 pixels, regardless of the quality parameter resolution.


/home/francesco/miniforge3/envs/deep-image-matching/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/francesco/miniforge3/envs/deep-image-matching/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Using coarse resolution (560, 560), and upsample res (860, 860)
2024-04-13 13:38:25 | [INFO    ] Running image matching with the following configuration:
2024-04-13 13:38:25 | [INFO    ]   Image folder: ../assets/example_cyprus/images
2024-04-13 13:38:25 | [INFO    ]   Output folder: ../assets/example_cyprus/results_roma_matching_lowres_quality_medium
2024-04-13 13:38:25 | [INFO    ]   Number of images: 10
2024-04-13 13:38:25 | [INFO    ]   Matching strategy: matching_lowres
2024-04-13 13:38:25 | [INFO    ]   Image quality: MEDIUM
2024-04-13 13:38:25 | [INFO    ]   Tile selection: NONE
2024-04-13 13:38:25 | [INFO    ]   Feature extraction method: no_extractor
2024-04-13 13:38:25 | [INFO    ]   Matching method: roma
2024-04-13 13:38:25 | [INFO    ]   Geometric verification: NONE
2024-04-13 13:38:25 | [INFO    ]   CUDA available: True
2024-04-13 13:38:25 | [INFO    ] Low resolution matching, generating pairs ..
Loaded SuperPoint model
2024-04-13 13:38:25 | [INFO    ] Extracting features 

100%|██████████| 10/10 [00:02<00:00,  4.64it/s]

2024-04-13 13:38:28 | [INFO    ] Matching downsampled images...


...

100%|██████████| 10/10 [00:00<00:00, 738.36it/s]

2024-04-13 13:38:29 | [INFO    ] Features extracted!
2024-04-13 13:38:29 | [INFO    ] Matching features with roma...
2024-04-13 13:38:29 | [INFO    ] roma configuration: 
{'coarse_res': 560,
 'name': 'roma',
 'num_sampled_points': 10000,
 'pretrained': 'outdoor',
 'upsample_res': 860}
2024-04-13 13:38:29 | [INFO    ] Matching features...
2024-04-13 13:38:29 | [INFO    ] 



  0%|          | 0/28 [00:00<?, ?it/s]/home/francesco/miniforge3/envs/deep-image-matching/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 28/28 [00:48<00:00,  1.73s/it]

2024-04-13 13:39:17 | [INFO    ] [Timer] | [matching] generate_pairs=9.582, extract_features=0.019, Match pair=1.726, Total execution=58.102
2024-04-13 13:39:17 | [INFO    ] [Timer] | [Deep Image Matching] Total execution=0.000
2024-04-13 13:39:17 | [WARNING ] Was not possible to load the first image to initialize cam0
2024-04-13 13:39:17 | [WARNING ] Was not possible to load the first image to initialize cam1



100%|██████████| 10/10 [00:00<00:00, 732.60it/s]
28it [00:00, 3828.29it/s]             


In [6]:
# Open sfm reconstruction with pycolmap
sfm_rec = pycolmap.Reconstruction(sfm_rec_path)

# Create a new database with the dense features and the known camera poses
db_from_existing_poses(
    dense_db_path,
    features_h5,
    matches_h5,
    sfm_rec_path,
    pair_file,
    do_geometric_verification=do_geometric_verification,
    max_error=max_error,
)

2024-04-13 13:39:28 | [WARNING ] The database already exists, deleting it.


Importing matches: 100%|██████████| 28/28 [00:00<00:00, 2145.23it/s]

2024-04-13 13:39:29 | [INFO    ] Performing geometric verification of the matches...



Importing verified matches: 100%|██████████| 7/7 [00:01<00:00,  3.66it/s]

2024-04-13 13:39:30 | [INFO    ] mean/med/min/max valid matches 59.78/55.11/39.00/97.69%.


In [7]:
# Run the triangulation with the known camera poses

# Define the options for the triangulation according to the IncrementalPipelineOptions available in pycolmap
# print(pycolmap.IncrementalPipelineOptions().summary())
opt = dict(
    triangulation=dict(
        ignore_two_view_tracks=False,
        min_angle=triang_min_angle,
    ),
)
verbose = True

with OutputCapture(verbose):
    with pycolmap.ostream():
        reconstruction = pycolmap.triangulate_points(
            sfm_rec,
            dense_db_path,
            image_dir,
            model_path,
            options=opt,
        )

I20240413 13:40:11.921245 3611342 misc.cc:198] 
Loading database
I20240413 13:40:11.922073 3611342 database_cache.cc:54] Loading cameras...
I20240413 13:40:11.922111 3611342 database_cache.cc:64]  1 in 0.000s
I20240413 13:40:11.922125 3611342 database_cache.cc:72] Loading matches...
I20240413 13:40:11.923058 3611342 database_cache.cc:78]  28 in 0.001s
I20240413 13:40:11.923079 3611342 database_cache.cc:94] Loading images...
I20240413 13:40:11.933820 3611342 database_cache.cc:143]  8 in 0.011s (connected 8)
I20240413 13:40:11.933871 3611342 database_cache.cc:154] Building correspondence graph...
I20240413 13:40:11.968505 3611342 database_cache.cc:190]  in 0.035s (ignored 0)
I20240413 13:40:11.970479 3611342 timer.cc:91] Elapsed time: 0.001 [minutes]
I20240413 13:40:11.976388 3611342 misc.cc:198] 
Triangulating image #1 (0)
I20240413 13:40:11.976423 3611342 sfm.cc:473] => Image sees 0 / 36298 points
I20240413 13:40:12.061582 3611342 sfm.cc:478] => Triangulated 35733 points
I20240413 13:4

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  3.111451e+05    0.00e+00    2.87e+02   0.00e+00   0.00e+00  1.00e+04        0    1.15e-01    4.58e-01
   1  3.078114e+05    3.33e+03    1.10e+00   2.16e+00   1.00e+00  3.00e+04        0    1.64e-01    6.23e-01
   2  3.078098e+05    1.58e+00    1.23e-01   3.82e+00   1.00e+00  9.00e+04        0    1.53e-01    7.76e-01


I20240413 13:40:13.397854 3611342 misc.cc:205] 
Bundle adjustment report
------------------------
I20240413 13:40:13.397927 3611342 bundle_adjustment.cc:942] 
    Residuals : 663988
   Parameters : 497991
   Iterations : 3
         Time : 0.811643 [s]
 Initial cost : 0.684544 [px]
   Final cost : 0.680865 [px]
  Termination : Convergence

I20240413 13:40:13.417588 3611342 incremental_mapper.cc:175] => Completed observations: 0
I20240413 13:40:13.432924 3611342 incremental_mapper.cc:178] => Merged observations: 0
I20240413 13:40:13.469981 3611342 incremental_mapper.cc:160] => Filtered observations: 13909
I20240413 13:40:13.470013 3611342 sfm.cc:521] => Changed observations: 0.041895
I20240413 13:40:13.609081 3611342 misc.cc:198] 
Bundle adjustment


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.766719e+05    0.00e+00    2.77e-04   0.00e+00   0.00e+00  1.00e+04        0    9.16e-02    3.94e-01


I20240413 13:40:14.188024 3611342 misc.cc:205] 
Bundle adjustment report
------------------------
I20240413 13:40:14.188091 3611342 bundle_adjustment.cc:942] 
    Residuals : 608352
   Parameters : 456264
   Iterations : 1
         Time : 0.423789 [s]
 Initial cost : 0.674381 [px]
   Final cost : 0.674381 [px]
  Termination : Convergence

I20240413 13:40:14.205947 3611342 incremental_mapper.cc:175] => Completed observations: 0
I20240413 13:40:14.218333 3611342 incremental_mapper.cc:178] => Merged observations: 0
I20240413 13:40:14.247723 3611342 incremental_mapper.cc:160] => Filtered observations: 0
I20240413 13:40:14.247749 3611342 sfm.cc:521] => Changed observations: 0.000000
I20240413 13:40:14.362433 3611342 misc.cc:198] 
Extracting colors


In [8]:
# Export the reconstruction in ply (to be opened in CloudCompare or Meshlab)
reconstruction.export_PLY(model_path / "model.ply")

# Export the reconstruction in bundler format (to be imported in Metashape)
reconstruction.export_bundler(
    model_path / "bundler.out",
    model_path / "bundler_list.txt",
    skip_distortion=True,
)

Now you can open the dense reconstruction also with the COLMAP GUI.
